In [21]:
import pystan
import pandas as pd
from bayesian_analysis import BayesianAnalysis, BayesianRevenue, BayesianConversion

In [22]:
pystan.__version__

'2.19.0.0'

In [23]:
sm_revenue = pystan.StanModel(file='revenue_model.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_e10b5aaf0bda8c3fd55c449a8894ff67 NOW.


In [4]:
boop = BayesianRevenue('data', 1, 1, sm_revenue)

In [5]:
df = pd.read_csv('../sevenpack/data/exp_385.csv')

In [13]:
unique_buckets = df.bucket.unique()

In [14]:
unique_buckets

array(['control', 'show_modal_forced_selection',
       'show_modal_optional_selection'], dtype=object)

In [15]:
results = []

In [16]:
results = []
for bucket in unique_buckets:
    bucket_df = df.loc[df["bucket"] == unique_buckets[0]]
    res = boop._generate_posterior_distribution(bucket_df)
    results.append(res)

In [20]:
len(results)

3

In [10]:
bucket_df = df.loc[df["bucket"] == unique_buckets[0]]
res = boop._generate_posterior_distribution(bucket_df)
results.append(res)

In [11]:
bucket_df = df.loc[df["bucket"] == unique_buckets[1]]
res = boop._generate_posterior_distribution(bucket_df)
results.append(res)

In [12]:
boop._generate_posterior_distribution_for_revenue(df)

AttributeError: 'BayesianRevenue' object has no attribute '_generate_posterior_distribution_for_revenue'

In [ ]:
df2 = pd.read_csv('../sevenpack/data/xp_432_cvr.csv')

In [ ]:
sm_conversion = pystan.StanModel(file = 'conversion_model.stan')

In [ ]:
boop2 = BayesianConversion('is_converted', 1, 1, sm_conversion)

In [ ]:
res = boop2._generate_posterior_distribution_for_conversions(df2)

In [ ]:
res

In [ ]:
sns.distplot(res)